**Nelder-Mead Method for Sinal Recovery**
<br>
*by: Cauã Gomes de Freitas and Nícolas Fagundes Ouverney*

# Initial Requirements

In [ ]:
# LIBRARIES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# DATABASE FILE PATH
file_path = r'db_pileup_noise.csv'

# DATABASE READING
db = pd.read_csv(file_path).values

# Definition of Functions

In [ ]:
# OBJECTIVE FUNCTION - ROOT MEAN-SQUARE ERROR
def objective_function(A):
    reconstruction_pattern = np.array([A[0]*1.0 + A[1]*0.4524 + A[2]*0.0172 + 0 + 0 + 0 + 0,
                      A[0]*0.5633 + A[1]*1 + A[2]*0.4524 + A[3]*0.0172 + 0 + 0 + 0,
                      A[0]*0.1493 + A[1]*0.5633 + A[2]*1 + A[3]*0.4524 + A[4]*0.0172 + 0 + 0,
                      A[0]*0.0424 + A[1]*0.1493 + A[2]*0.5633 + A[3]*1 + A[4]*0.4524 + A[5]*0.0172 + 0,
                      0 + A[1]*0.0424 + A[2]*0.1493 + A[3]*0.5633 + A[4]*1 + A[5]*0.4524 + A[6]*0.0172,
                      0 + 0 + A[2]*0.0424 + A[3]*0.1493 + A[4]*0.5633 + A[5]*1 + A[6]*0.4524,
                      0 + 0 + 0 + A[3]*0.0424 + A[4]*0.1493 + A[5]*0.5633 + A[6]*1], dtype = np.longdouble)
    aux_sum = data_signal - reconstruction_pattern
    return (sum(aux_sum ** 2) / len(data_signal)) ** 0.5

In [ ]:
# NELDER-MEAD METHOD IMPLEMENTATION
def Nelder_Mead_Simplex(f, initial_points, max_iter, tol1, tol2):
    n = len(initial_points) # number of dimensions of the function f
    # Adaptive Nelder-Mead Simplex
    alpha = 1 # reflection coefficient
    gamma = 1 + 2/n # expansion coefficient
    rho = 0.75 - 1/(2*n) # contraction coefficient
    sigma = 1 - 1/n # shrink coefficient

    # defines points for a initial simplex
    simplex = [initial_points]    
    for i in range(0, n):
        aux_init_simplex = initial_points[:]
        for j in range(0, n):
            if i == j:
                aux_init_simplex[i] += 0.05
        simplex.append(np.array(aux_init_simplex))
    
    # preparation for the loop
    k = 0
    while k < max_iter or max_iter == -1:
        simplex.sort(key = lambda x: f(x))
        best_value = f(simplex[0])
        worst_value = f(simplex[-1])

        # check the objetive function value and the interval
        if abs(best_value) < tol1 and abs(f(simplex[0]) - f(simplex[-1])) < tol2:
            break
        
        k += 1
        centroid = np.mean(simplex[:-1], dtype = np.longdouble, axis = 0) # axis = 0 for efetuate the mean along the column
        
        # calculates reflection point
        reflected_point = centroid + alpha * (centroid - simplex[-1])
        reflected_value = f(reflected_point)

        # reflection
        if reflected_value >= best_value and reflected_value < f(simplex[-2]):
            simplex[-1] = reflected_point[:]
        
        # expansion
        elif reflected_value < best_value:
            expansion_point = centroid + gamma * (reflected_point - centroid)
            expansion_value = f(expansion_point)

            if expansion_value < reflected_value:
                simplex[-1] = expansion_point[:]
            else:
                simplex[-1] = reflected_point[:]
        
        # contraction
        else:
            if reflected_value < worst_value:
                contracted_point = centroid + rho * (reflected_point - centroid)

                if f(contracted_point) < reflected_value:
                    simplex[-1] = contracted_point[:]
                # shrink
                else:
                    for j in range(1, n + 1):
                        simplex[j] = simplex[0] + sigma * (simplex[j] - simplex[0])
            else:
                contracted_point = centroid + rho * (simplex[-1] - centroid)
                if f(contracted_point) < worst_value:
                    simplex[-1] = contracted_point[:]
                # shrink
                else:
                    for j in range(1, n + 1):
                        simplex[j] = simplex[0] + sigma * (simplex[j] - simplex[0])
    
    simplex.sort(key = lambda x: f(x))

    return simplex[0], k

# Inputs and Execution of the Method

In [ ]:
# INPUTS
# stopping criteria
max_iterations = 10000 # put -1 to ignore
tolerance_1 = 1e-12 # this tolerance is calculated by the absolute value of the objective function 
tolerance_2 = 1e-12 # this tolerance is calculated by the differecence between the best and worst function value calculated at the point

# initial simplex
initial_guess = [1000, 1000, 1000, 1000, 1000, 1000, 1000]

In [ ]:
# FUNCTION CALLING
mean_residuals = []
all_residuals = []
points = []
points_amplitude = []
average_iterations = 0
for i in range(0, len(db), 8):
    # reading the stacking row from database
    data_signal = db[i + 7, 2 :]

    # calling Nelder-Mead Simplex
    best_point, num_iter = Nelder_Mead_Simplex(objective_function, initial_guess, max_iterations, tolerance)

    # saving datas for the results
    points.append(best_point)
    average_iterations += num_iter
    if len(points) % 100 == 0:
        print('Progress: {}/{}'.format(len(points), len(db)//8))

    # calculating the average error of the amplitudes
    amplitudes = list(map(np.longdouble, db[i : i + 7, 1]))
    points_amplitude.append(amplitudes)
    all_residuals.append(amplitudes - best_point)
    mean_residuals.append(np.mean(amplitudes - best_point, dtype = np.longdouble))
average_iterations = average_iterations / len(points)

# Results

In [ ]:
# SHOWING THE RESULTS
print('Mean Residuals:')
print(mean_residuals)
print('Average of Iterations:')
print(average_iterations)
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (9, 9), tight_layout = True)
count = 0
for i in range(0, 3):
    for j in range(0, 3):
        if i == 0 and j == 0:
            ax[i, j].hist(mean_residuals, bins = 200)
            ax[i, j].set_title('MEAN RESIDUALS')
            ax[i, j].set_xlabel('Residual')
            ax[i, j].set_ylabel('Volume')
        elif i == 0 and j == 1:
            aux_residuals = []
            for k in range(0, len(all_residuals)):
                for l in range(0, len(all_residuals[0])):
                    aux_residuals.append(all_residuals[k][l])
            ax[i, j].hist(aux_residuals, bins = 200)
            ax[i, j].set_title('ALL RESIDUALS')
            ax[i, j].set_xlabel('Residual')
            ax[i, j].set_ylabel('Volume')
        else:
            by_time = aux_residuals[count::7]
            count += 1
            ax[i, j].hist(by_time, bins = 200)
            ax[i, j].set_title(f'RESIDUALS TIME {count}')
            ax[i, j].set_xlabel('Residual')
            ax[i, j].set_ylabel('Volume')
            
plt.show()

In [ ]:
# REAL AND CALCULATED AMPLITUDES
print('CALCULATED AMPLITUDES')
for i in range(0, len(points)):
    print('Before pile_up_{}:'.format(i + 1))
    print(list(points[i]))
    print(list(points_amplitude[i]),'\n')